## Setup of environnement
This section contains all the imports of modules that are required to run this Notebook.

In [1]:
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import col, explode, json_tuple, regexp_replace
from pyspark.sql.functions import sum as col_sum
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, ArrayType, FloatType
import pyspark
from pyspark import SparkConf

import re
import os

## The amount of citizens per city
In this section the citizens file is read and converted to a Pyspark dataframe.
With this dataframe, calculations on the data can be made easier.
First, the unneccessary headers are removed from the data.
The French equivalent of the town names is not important and is thus removed.
Empty lines and excess whitespaces are removed as well.
The numbers in this file contain a '.' if the numbers are larger than 999.
This '.' is removed as well.
The output of this section are the first 20 entries of the resulting dataframe and the schema that is used within the dataframe.

In [2]:
citizens = sc.textFile("data/citizens.txt")
citizens.collect()

# Remove unnecessary headers
citizens = citizens.map(lambda x: re.sub(
    r'^KONINKRIJK.*|^BRUSSELS.*|^ARR.*|^ARRONDISSEMENT.*|^PROVINC.*|^VLAAMS.*|^REGION.*', '', x))

# Replace 'village / village-in-french' with 'village'
citizens = citizens.map(lambda x: re.sub(r'/ .* ', '', x))

# Remove everything in between parentheses
citizens = citizens.map(lambda x: re.sub(r'\(.*\)', '', x))

# Remove excess whitespaces
citizens = citizens.map(lambda x: re.sub(r"[^\S\n\t]+", ' ', x))

# Remove empty lines
citizens = citizens.filter(lambda x: x != '')

# Remove '.' from the numbers
citizens = citizens.map(lambda x: re.sub(r'\.', '', x))

# Split on space: gives a list of lists [[village, amount], [village, amount]...]
citizens = citizens.map(lambda x: x.rsplit(' ', 1))


# Create schema for dataframe
s = StructType([
    StructField('village', StringType(), False),
    StructField('citizens', StringType(), False)
])

# Create schema and cast the citizens column to IntegerType
citizens = citizens.toDF(schema=s)

# Cast to citizens column to integers
citizens = citizens.withColumn("citizens", citizens['citizens'].cast(IntegerType()))

    
citizens.show()
citizens.printSchema()

+--------------------+--------+
|             village|citizens|
+--------------------+--------+
|          Anderlecht|  117724|
|             Brussel|  177112|
|              Elsene|   86336|
|           Etterbeek|   47410|
|               Evere|   41016|
|           Ganshoren|   24794|
|               Jette|   52144|
|          Koekelberg|   21765|
|            Oudergem|   33725|
|          Schaarbeek|  132097|
| Sint‐Agatha‐Berchem|   24831|
|         Sint‐Gillis|   49361|
| Sint‐Jans‐Molenbeek|   95455|
| Sint‐Joost‐ten‐Node|   26813|
|Sint‐Lambrechts‐W...|   56212|
| Sint‐Pieters‐Woluwe|   41513|
|               Ukkel|   82038|
|               Vorst|   55694|
| Watermaal‐Bosvoorde|   25001|
|          Aartselaar|   14298|
+--------------------+--------+
only showing top 20 rows

root
 |-- village: string (nullable = false)
 |-- citizens: integer (nullable = true)



## Stops
In this section the stops txt file gets converted into a usable dataframe.
The output of this section are the first 20 entries of the resulting dataframe.
Village number, entity number and the links irrelevant data, so they are removed from the dataframe.
To be more consistent with the rest of this notebook, the Dutch names are translated to English names.

In [3]:
# Read in the file (as json) and convert to a single column
# The 'haltes' column gets renamed to 'stops'
stops = spark.read.json("data/stops.txt")
stops = stops.select((explode("haltes").alias("stops")))

# Map each entry in the dataframe to its own stop
stops = stops.select('stops').rdd.map(lambda x: x.stops).toDF()
# Drop unnecessary data
stops = stops.drop('links', 'gemeentenummer', 'entiteitnummer')

# Rename columns of dataframe to better name
stops = stops \
            .withColumnRenamed('haltenummer', 'stop_number') \
            .withColumnRenamed('omschrijving', 'desc') \
            .withColumnRenamed('geoCoordinaat', 'coord') \
            .withColumnRenamed('omschrijvingGemeente', 'vill') \

# Get the amount of stops per town
per_town = stops.groupBy("vill").count().select(col("vill"), col("count").alias("# stops"))
per_town.show()

+-------------------+-------+
|               vill|# stops|
+-------------------+-------+
|              Bever|     28|
|           Waanrode|     40|
|              Duras|      2|
|           Ettelgem|      8|
|          Zillebeke|     25|
|Sint-Job-in-'t-Goor|     24|
|               Lint|     24|
|          Merelbeke|     85|
|             Essene|     31|
|             Parike|     10|
|          Huizingen|     21|
| Glabbeek-Zuurbemde|     16|
|             Gorsem|      6|
|          Harelbeke|     81|
|                Mol|    140|
|        Hoogstraten|     38|
|       Oud-Turnhout|     47|
|           Hoevenen|     24|
|         Rupelmonde|      6|
|            Edingen|     12|
+-------------------+-------+
only showing top 20 rows



## Combining the dataframes
The next section combines the citizens dataframe with the dataframe that contains the amount of citizens per village.
By doing so, it is easy to calculate the amount of stops per citizen of each village.

In [4]:
data = per_town.join(citizens, citizens.village == per_town.vill)
data = data.drop('vill')
data.show()
data.printSchema()

+-------+-------------+--------+
|# stops|      village|citizens|
+-------+-------------+--------+
|     28|        Bever|    2205|
|     81|    Harelbeke|   27879|
|     24|         Lint|    8776|
|     85|    Merelbeke|   24629|
|     17|   Alveringem|    5087|
|     38|  Hoogstraten|   21293|
|    140|          Mol|   36506|
|     60|     Oostkamp|   23577|
|      4|        Oreye|    3916|
|     53|   Aartselaar|   14298|
|     30|   Huldenberg|    9882|
|     98|      Torhout|   20504|
|     48|   Liedekerke|   13181|
|     47|   Wuustwezel|   20663|
|     25|     Kruibeke|   16661|
|    129|     Overijse|   25169|
|     30|    Boutersem|    8165|
|    159|   Diepenbeek|   19133|
|     44|    Houthulst|   10033|
|     71|Nieuwerkerken|    6957|
+-------+-------------+--------+
only showing top 20 rows

root
 |-- # stops: long (nullable = false)
 |-- village: string (nullable = false)
 |-- citizens: integer (nullable = true)



## Calculating the amount of stops per citizen of each village
This final section uses the dataframe created in the previous section to calculate the amount of stops per citizen of each village. The resulting dataframe is sorted by the result in descending order.
Unnecessary data is removed so that the output is as clean as possible.
The output only shows the first 20 entries of the dataframe.

In [6]:
# Add a column that contains the result and sort the table by this column
result = data.withColumn("Result", col("# stops")/col("citizens")).sort(col('Result').desc())
# Drop unnecessary data
result = result.drop('# stops', 'citizens')
# Show the result
result.show()

+-------------+--------------------+
|      village|              Result|
+-------------+--------------------+
|    Herstappe| 0.06818181818181818|
|        Bever|0.012698412698412698|
|     Geetbets|0.010941644562334218|
|Nieuwerkerken|0.010205548368549663|
|        Alken| 0.01020408163265306|
|   Wachtebeke|0.008989056800416884|
|     Overpelt|0.008533747090768037|
|   Diepenbeek|0.008310249307479225|
|     Pepingen| 0.00800182898948331|
|   Zuienkerke|0.007686676427525...|
|       Wellen|0.007296311309282529|
|   Hoegaarden|0.006772334293948127|
|    Koekelare|0.006709882861366996|
|    Ruiselede|0.006685236768802228|
| Begijnendijk|0.006664677210782851|
|        Herne| 0.00662451068955134|
|         Bree|           0.0061875|
|        Balen|0.006109253065774805|
|      Berlaar|0.006083782374413...|
|        Melle|0.006051698798305...|
+-------------+--------------------+
only showing top 20 rows

